In [1]:
import pandas as pd

In [2]:
#real excel
xls = pd.ExcelFile('C:/Users/CHA82870/Desktop/2017dolphins/3RSET_CWD_Database.xlsx')

## Prepare Sample Table

In [3]:
effort = xls.parse('Survey Effort')

In [4]:
#remove space in header
effort = effort.rename(columns=lambda x: x.strip())

#remove space in content
df_obj = effort.select_dtypes(['object'])
df_obj
effort[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [5]:
effort.head()

,DATE,AREA,BEAU,KM SEARCHED,SEASON,VESSEL,TYPE
0,2015-12-18,AW,3,5.45,WINTER,32166,3RS ET BASELINE
1,2015-12-18,WL,3,15.50,WINTER,32166,3RS ET BASELINE
2,2015-12-18,WL,4,18.74,WINTER,32166,3RS ET BASELINE
3,2015-12-18,SWL,2,0.29,WINTER,32166,3RS ET BASELINE
4,2015-12-18,SWL,3,3.56,WINTER,32166,3RS ET BASELINE


In [6]:
#filter relevant time for analysis
effort = effort[(effort['DATE']>= '2017-1-1') & (effort['DATE']<= '2017-12-31')]

effort = effort[effort['BEAU'] <= 3]

effort = effort[effort['AREA'] != 'DB' ] 

In [7]:
effort.head()

,DATE,AREA,BEAU,KM SEARCHED,SEASON,VESSEL,TYPE
439,2017-01-05,AW,2,4.860,WINTER,32166,3RS ET
440,2017-01-05,WL,1,12.529,WINTER,32166,3RS ET
441,2017-01-05,WL,2,14.382,WINTER,32166,3RS ET
442,2017-01-05,SWL,2,6.010,WINTER,32166,3RS ET
443,2017-01-06,SWL,1,1.300,WINTER,32166,3RS ET


In [8]:
sample_table = {'Sample.Label': [1,2,3,4,5], 'Region.Label':['NWL','SWL','AW','NEL','WL'], 'Effort':[effort['KM SEARCHED'][effort['AREA'] == 'NWL'].sum(), effort['KM SEARCHED'][effort['AREA'] == 'SWL'].sum(),effort['KM SEARCHED'][effort['AREA'] == 'AW'].sum(), effort['KM SEARCHED'][effort['AREA'] == 'NEL'].sum(),effort['KM SEARCHED'][effort['AREA'] == 'WL'].sum()]}

In [9]:
sample_table = pd.DataFrame(sample_table, columns = ['Sample.Label','Region.Label','Effort'])

In [10]:
sample_table.head()

,Sample.Label,Region.Label,Effort
0,1,NWL,1535.881
1,2,SWL,1299.606
2,3,AW,109.789
3,4,NEL,1005.510
4,5,WL,632.956


## Prepare Region Table

In [11]:
region_table = {'Region.Label':['NWL', 'SWL', 'AW', 'NEL', 'WL'], 'Area':[87.415,66.902,2.170,52.190,32.649]}

In [12]:
region_table = pd.DataFrame(region_table, columns = ['Region.Label','Area'])

In [13]:
region_table.head()

,Region.Label,Area
0,NWL,87.415
1,SWL,66.902
2,AW,2.170
3,NEL,52.190
4,WL,32.649


## Prepare obs table

In [14]:
#access sighting sheet
sighting = xls.parse('Sightings')
#remove space in header
sighting = sighting.rename(columns=lambda x: x.strip())
#remove space in content
df_obj = sighting.select_dtypes(['object'])
df_obj
sighting[df_obj.columns] = df_obj.astype("str").apply(lambda x: x.str.strip())
#filter date, CWD, BEAU, On Effort
sighting = sighting[(sighting['DATE'] >= '2017-1-1')& (sighting['DATE'] <= '2017-12-31')]
sighting = sighting[sighting['CWD/FP'] == 'CWD']
sighting = sighting[sighting['BEAU'] <= 3]
sighting = sighting[sighting['EFFORT'] == 'ON']
sighting = sighting[sighting['AREA'] != 'DB' ] 

In [15]:
sighting.head()

,DATE,STG #,TIME,CWD/FP,HRD SZ,AREA,BEAU,PSD,EFFORT,SEEN BY,TYPE,DEC LAT,DEC LON,GRID,SEASON,BOAT ASSOC.,BEHAVIOUR,Remarks,ID INDIVIDUALS,GROUP COMPOSITION
253,2017-01-05,1,1010,CWD,9,WL,2,822.0,ON,JT,3RS ET,22.293417,113.861194,19E,WINTER,NONE,TRAVELING / PORPOISING,MOTHER & UJ,"NLMM015, NLMM016, WLMM064, WLMM065, WLMM066, S...","UJ:2, SJ:1, SS:1 SA:3"
254,2017-01-05,2,1051,CWD,2,WL,2,1361.0,ON,JT,3RS ET,22.273806,113.848222,22D,WINTER,NONE,BREACHING,nan,"WLMM067, WLMM068","SS:1, SA:1"
255,2017-01-05,3,1118,CWD,2,WL,2,118.0,ON,VW,3RS ET,22.258361,113.838139,23C,WINTER,NONE,BREACHING / SPY-HOPPING,nan,nan,nan
256,2017-01-05,4,1150,CWD,1,WL,2,65.0,ON,JT,3RS ET,22.241278,113.833944,25B,WINTER,NONE,UNDETERMINED,nan,nan,nan
257,2017-01-05,5,1208,CWD,1,WL,2,86.0,ON,JT,3RS ET,22.232111,113.831583,26B,WINTER,NONE,UNDETERMINED,nan,SLMM010,SS:1


In [16]:
#sighitng table can be further cut
sighting = sighting[['DATE', 'HRD SZ', 'AREA','PSD']]
sighting.head()

,DATE,HRD SZ,AREA,PSD
253,2017-01-05,9,WL,822.0
254,2017-01-05,2,WL,1361.0
255,2017-01-05,2,WL,118.0
256,2017-01-05,1,WL,65.0
257,2017-01-05,1,WL,86.0


In [17]:
#rename the columns name to distance program format
sighting.columns = ['DATE', 'size', 'Region.Label', 'distance']
sighting.head()

,DATE,size,Region.Label,distance
253,2017-01-05,9,WL,822.0
254,2017-01-05,2,WL,1361.0
255,2017-01-05,2,WL,118.0
256,2017-01-05,1,WL,65.0
257,2017-01-05,1,WL,86.0


## Prepare fiat table

In [18]:
fiat = pd.merge(sighting, sample_table, on=['Region.Label'], how='left')
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort
0,2017-01-05,9,WL,822.0,5,632.956
1,2017-01-05,2,WL,1361.0,5,632.956
2,2017-01-05,2,WL,118.0,5,632.956
3,2017-01-05,1,WL,65.0,5,632.956
4,2017-01-05,1,WL,86.0,5,632.956


In [19]:
fiat = pd.merge(fiat, region_table, on=['Region.Label'],how='left')

In [20]:
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
0,2017-01-05,9,WL,822.0,5,632.956,32.649
1,2017-01-05,2,WL,1361.0,5,632.956,32.649
2,2017-01-05,2,WL,118.0,5,632.956,32.649
3,2017-01-05,1,WL,65.0,5,632.956,32.649
4,2017-01-05,1,WL,86.0,5,632.956,32.649


In [21]:
fiat['distance'] = fiat['distance']/1000

In [22]:
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
0,2017-01-05,9,WL,0.822,5,632.956,32.649
1,2017-01-05,2,WL,1.361,5,632.956,32.649
2,2017-01-05,2,WL,0.118,5,632.956,32.649
3,2017-01-05,1,WL,0.065,5,632.956,32.649
4,2017-01-05,1,WL,0.086,5,632.956,32.649


In [23]:
fiat.to_csv('C:/Users/CHA82870/Desktop/2017dolphins//fiat_before_sorting.csv')

In [24]:
fiat = fiat.sort_values(by = ['Region.Label'])
fiat.head()

,DATE,size,Region.Label,distance,Sample.Label,Effort,Area
143,2017-08-22,1,AW,0.087,3,109.789,2.17
206,2017-12-07,5,AW,0.145,3,109.789,2.17
14,2017-01-19,3,AW,0.023,3,109.789,2.17
22,2017-02-16,2,AW,0.016,3,109.789,2.17
198,2017-11-21,5,AW,0.046,3,109.789,2.17


In [25]:
fiat.to_csv('C:/Users/CHA82870/Desktop/2017dolphins/fiat2.txt')
fiat.to_csv('C:/Users/CHA82870/Desktop/2017dolphins/fiat2.csv')